# Importing libraries and loading in our datasets

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
plt.style.use('ggplot')

In [3]:
holes = pd.read_pickle('holes.pkl')
wk26 = pd.read_pickle('26.pkl')
wk12 = pd.read_pickle('12.pkl')
wk1 = pd.read_pickle('7.pkl')
DOS7 = pd.read_pickle('7DOS.pkl')

for i in list([wk26, wk12, wk1, DOS7]):
    print(i.dtypes)

Item Code                    int64
Product                     object
UPC                          int64
Bill Units                   int64
Scan Units                   int64
Est. Units On Hand           int64
Est. Sell Thru % (Units)     int64
Bill $                       int64
Sales                        int64
dtype: object
Item Code                    int64
Product                     object
UPC                          int64
Bill Units                   int64
Scan Units                   int64
Est. Units On Hand           int64
Est. Sell Thru % (Units)     int64
Bill $                       int64
Sales                        int64
dtype: object
Item Code                    int64
Product                     object
UPC                          int64
Bill Units                   int64
Scan Units                   int64
Est. Units On Hand           int64
Est. Sell Thru % (Units)     int64
Bill $                       int64
Sales                        int64
dtype: object
Fiscal Date  

In [4]:
holes

,UPC,Product
0,1780014500,CAT CHOW NATURALS
1,82927421880,MM BASTED BITES CKN TUNA
2,82927451250,MEOW MIX TENDERCENTRS SLMN-CKN
3,5000050037,FRISKIES PARTY MIX BEACH 30OZ
4,5000023891,FRISKIES PARTY MIX ORIGINAL
5,7023011620,TIDY CATS 24/7 JUG LITTER PRFM
6,7023015345,TIDY CATS GLADE
7,5000057797,FRISKIES TASTY TREAS OCEANFISH
8,5000051420,FANCY FEAST SEAFOOD VP 36CT
9,2310011834,SHEBA PP CHKN TUNA


The data appears to be in tact. Since we're going to wind up joining these tables along the UPC, which is a unique identifier, I want to bring in the function from before which will help identify what's missing between two sets. This will help me anticipate what our set will look like.

In [5]:
def UPC_check(big, small):
    check = []
    for i in big.UPC.unique():
        if i in small.UPC.unique():
            continue
        else:
            check.append(i)
    return(check)

In [6]:
check = UPC_check(holes, wk26)
print(check)

[5000023891, 7119000778, 85645200503, 2310010495]


In [7]:
holes.loc[holes.UPC.isin(check)]

,UPC,Product
4,5000023891,FRISKIES PARTY MIX ORIGINAL
10,7119000778,RR NUTRISH OCEAN LOVERS VP
15,85645200503,BUCKLEY SKIN & COAT CHICKEN
42,2310010495,CESAR SAVORY DELIGHTS


In [8]:
wk26.loc[wk26.UPC.isin(check)]

,Item Code,Product,UPC,Bill Units,Scan Units,Est. Units On Hand,Est. Sell Thru % (Units),Bill $,Sales


It looks like our list of Holes have four UPCs that have no sales history in 6 months. When we do a left join to the largest data set, it means these UPCs will get dropped. This is probably for the best - UPCs that have no sales history in six months probably represent issues at a store level, but, more importantly, with no sales data there's nothing for our model to use to predict.

It should be okay that these get washed out.

In [9]:
check = UPC_check(holes, wk12)
print(check)

[5000023891, 7119000778, 85645200503, 2310010495, 750151480001]


In [10]:
holes.loc[holes.UPC.isin(check)]

,UPC,Product
4,5000023891,FRISKIES PARTY MIX ORIGINAL
10,7119000778,RR NUTRISH OCEAN LOVERS VP
15,85645200503,BUCKLEY SKIN & COAT CHICKEN
42,2310010495,CESAR SAVORY DELIGHTS
48,750151480001,NUMI CAT LITTER


In [11]:
wk26.loc[wk26.UPC.isin(check)]

,Item Code,Product,UPC,Bill Units,Scan Units,Est. Units On Hand,Est. Sell Thru % (Units),Bill $,Sales
962,557211,NUMI CAT LITTER,750151480001,0,0,0,-1,0,0


This checked our holes w/out a 12 week sales history. Of those items without a 12 week history, they all had at least a 6 month sales history, except for one. So, I don't need to worry about this when I join the datasets.

In [12]:
parent = wk26.copy()
parent.head()

,Item Code,Product,UPC,Bill Units,Scan Units,Est. Units On Hand,Est. Sell Thru % (Units),Bill $,Sales
0,357090,ALPO SNAPS VARIETY PK DOG TRTS,1113200392,240,237,3,99,638,631
1,336347,ALPO PRIME CUTS W/ BEEF,1113212556,86,84,2,98,412,400
2,59442,ALPO CHOPHOUSE T-BNE STK GRAVY,1113213597,468,490,-22,105,365,382
3,164820,ALPO PRIME CUTS,1113214544,64,63,1,98,511,504
4,298380,ONE CAT HAIRBALL FORMULA,1780001260,44,45,-1,102,505,517


In [13]:
parent.drop(labels = ['Item Code', 'Product'], axis = 1, inplace = True)
parent.head()

,UPC,Bill Units,Scan Units,Est. Units On Hand,Est. Sell Thru % (Units),Bill $,Sales
0,1113200392,240,237,3,99,638,631
1,1113212556,86,84,2,98,412,400
2,1113213597,468,490,-22,105,365,382
3,1113214544,64,63,1,98,511,504
4,1780001260,44,45,-1,102,505,517


In [14]:
parent.rename(columns = {'Bill Units': 'Bill Units 26',
                         'Scan Units': 'Scan Units 26',
                         'Est. Units On Hand': 'UOH 26',
                         'Est. Sell Thru % (Units)': 'Sell Thru 26',
                         'Bill $': 'Bill 26',
                         'Sales': 'Sales 26'}, inplace = True)
parent.head()

,UPC,Bill Units 26,Scan Units 26,UOH 26,Sell Thru 26,Bill 26,Sales 26
0,1113200392,240,237,3,99,638,631
1,1113212556,86,84,2,98,412,400
2,1113213597,468,490,-22,105,365,382
3,1113214544,64,63,1,98,511,504
4,1780001260,44,45,-1,102,505,517


In [15]:
parent = parent.merge(wk12, how = 'left', on = 'UPC')
parent.rename(columns = {'Bill Units': 'Bill Units 12',
                         'Scan Units': 'Scan Units 12',
                         'Est. Units On Hand': 'UOH 12',
                         'Est. Sell Thru % (Units)': 'Sell Thru 12',
                         'Bill $': 'Bill 12',
                         'Sales': 'Sales 12'}, inplace = True)
parent.head()

,UPC,Bill Units 26,Scan Units 26,UOH 26,Sell Thru 26,Bill 26,Sales 26,Item Code,Product,Bill Units 12,Scan Units 12,UOH 12,Sell Thru 12,Bill 12,Sales 12
0,1113200392,240,237,3,99,638,631,357090.0,ALPO SNAPS VARIETY PK DOG TRTS,104.0,105.0,-1.0,101.0,277.0,280.0
1,1113212556,86,84,2,98,412,400,336347.0,ALPO PRIME CUTS W/ BEEF,46.0,45.0,1.0,98.0,220.0,213.0
2,1113213597,468,490,-22,105,365,382,59442.0,ALPO CHOPHOUSE T-BNE STK GRAVY,168.0,197.0,-29.0,117.0,131.0,154.0
3,1113214544,64,63,1,98,511,504,164820.0,ALPO PRIME CUTS,29.0,29.0,0.0,100.0,232.0,232.0
4,1780001260,44,45,-1,102,505,517,298380.0,ONE CAT HAIRBALL FORMULA,16.0,16.0,0.0,100.0,184.0,184.0


In [16]:
parent.isnull().sum()

UPC              0
Bill Units 26    0
Scan Units 26    0
UOH 26           0
Sell Thru 26     0
Bill 26          0
Sales 26         0
Item Code        9
Product          9
Bill Units 12    9
Scan Units 12    9
UOH 12           9
Sell Thru 12     9
Bill 12          9
Sales 12         9
dtype: int64

In [17]:
parent.loc[parent['Product'].isnull()]

,UPC,Bill Units 26,Scan Units 26,UOH 26,Sell Thru 26,Bill 26,Sales 26,Item Code,Product,Bill Units 12,Scan Units 12,UOH 12,Sell Thru 12,Bill 12,Sales 12
151,1901480137,0,1,-1,-1,0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167,2027999502,12,9,3,75,66,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
442,4122067532,2,1,1,50,12,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
443,4122067533,3,3,0,100,17,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
739,7910051446,0,1,-1,-1,0,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
746,7910052703,0,1,-1,-1,0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
898,84024312257,0,3,-3,-1,0,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,85736400406,0,2,-2,-1,0,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
962,750151480001,0,0,0,-1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We are going to have to make decisions about how to handle these values, since we'll only continue to run into more null values as we go along. Items with a history in the last six months won't necessarily sell units in one fiscal week, or one sales day. Since BI will only populate values for which there is data, it's up to us to determine how to fill this data in.

1. **Item Code**  is a field we're going to drop. Same for **Product**.
2. **Bill Units, Scan Units, Bill, Sales** should all be set to 0, since we're selling and ordering zero of these UPCs in this time frame.
3. **Sell Thru** should also be set to 0. So far, in the previous clean we set Sell Thru to be -1 when there was an 'error' with the math that produced the variable. In those cases, it could be due to a zero denominator even though there's >= 1 unit sold. So, if we use that same stand in we're going to mess up the integrity of our data.

### UOH - Units on Hand

This is the tricky one. UOH is supposed to be the store's count of how much inventory it actually has. The actual UOH isn't available to us to query, so BI estimates it from histories of orders and sales.

The reason we have a 6mo history in this set is to get an approximation of what the true UOH estimate is. The short term estimated UOH is less practical from an operations stand point, but we're going to set this to 0 in cases where it's NAN.

In [18]:
parent.drop(['Item Code', 'Product'], inplace = True, axis = 1)
parent.fillna(value = 0, inplace = True)
parent.isnull().sum()

UPC              0
Bill Units 26    0
Scan Units 26    0
UOH 26           0
Sell Thru 26     0
Bill 26          0
Sales 26         0
Bill Units 12    0
Scan Units 12    0
UOH 12           0
Sell Thru 12     0
Bill 12          0
Sales 12         0
dtype: int64

In [19]:
parent = parent.merge(wk1, how = 'left', on = 'UPC')
parent.rename(columns = {'Bill Units': 'Bill Units 7',
                         'Scan Units': 'Scan Units 7',
                         'Est. Units On Hand': 'UOH 7',
                         'Est. Sell Thru % (Units)': 'Sell Thru 7',
                         'Bill $': 'Bill 7',
                         'Sales': 'Sales 7'}, inplace = True)
parent.head()

,UPC,Bill Units 26,Scan Units 26,UOH 26,Sell Thru 26,Bill 26,Sales 26,Bill Units 12,Scan Units 12,UOH 12,...,Bill 12,Sales 12,Item Code,Product,Bill Units 7,Scan Units 7,UOH 7,Sell Thru 7,Bill 7,Sales 7
0,1113200392,240,237,3,99,638,631,104.0,105.0,-1.0,...,277.0,280.0,357090.0,ALPO SNAPS VARIETY PK DOG TRTS,4.0,10.0,-6.0,250.0,11.0,27.0
1,1113212556,86,84,2,98,412,400,46.0,45.0,1.0,...,220.0,213.0,336347.0,ALPO PRIME CUTS W/ BEEF,6.0,6.0,0.0,100.0,29.0,27.0
2,1113213597,468,490,-22,105,365,382,168.0,197.0,-29.0,...,131.0,154.0,59442.0,ALPO CHOPHOUSE T-BNE STK GRAVY,36.0,12.0,24.0,33.0,28.0,9.0
3,1113214544,64,63,1,98,511,504,29.0,29.0,0.0,...,232.0,232.0,164820.0,ALPO PRIME CUTS,3.0,4.0,-1.0,133.0,24.0,32.0
4,1780001260,44,45,-1,102,505,517,16.0,16.0,0.0,...,184.0,184.0,298380.0,ONE CAT HAIRBALL FORMULA,0.0,2.0,-2.0,-1.0,0.0,23.0


In [20]:
parent.drop(['Item Code', 'Product'], inplace = True, axis = 1)
parent.fillna(value = 0, inplace = True)
parent.isnull().sum()

UPC              0
Bill Units 26    0
Scan Units 26    0
UOH 26           0
Sell Thru 26     0
Bill 26          0
Sales 26         0
Bill Units 12    0
Scan Units 12    0
UOH 12           0
Sell Thru 12     0
Bill 12          0
Sales 12         0
Bill Units 7     0
Scan Units 7     0
UOH 7            0
Sell Thru 7      0
Bill 7           0
Sales 7          0
dtype: int64

# Adding in the most recent 7 days of sales, individually

Something that is useful on the sales floor when you're trying to make a decision is to see how long ago the last sale was made and contextualize it in terms of the week. Sometimes big gaps in sales mean there's an issue that's gone unnoticed for a long time. This is a feature included in SRS that simply isn't available through BI, so I have to attempt to re-create it through what's available.

In [21]:
DOS7.head()

,Fiscal Date,Item Code,Product,UPC,Bill Units,Scan Units,Est. Units On Hand,Est. Sell Thru % (Units),Bill $,Sales
0,1/31/2020,357090,ALPO SNAPS VARIETY PK DOG TRTS,1113200392,0,2,-2,-1,0,5
1,1/31/2020,336347,ALPO PRIME CUTS W/ BEEF,1113212556,0,2,-2,-1,0,10
2,1/31/2020,59442,ALPO CHOPHOUSE T-BNE STK GRAVY,1113213597,12,0,12,0,9,0
3,1/31/2020,357071,ONE CAT HAIRBALL FORMULA,1780001263,0,2,-2,-1,0,16
4,1/31/2020,357088,ONE CAT SENIOR FORUMLA,1780001885,4,0,4,0,31,0


In [22]:
DOS7['Fiscal Date'] = pd.to_datetime(DOS7['Fiscal Date'])

In [23]:
DOS7.dtypes

Fiscal Date                 datetime64[ns]
Item Code                            int64
Product                             object
UPC                                  int64
Bill Units                           int64
Scan Units                           int64
Est. Units On Hand                   int64
Est. Sell Thru % (Units)             int64
Bill $                               int64
Sales                                int64
dtype: object

In [24]:
DOS7['Fiscal Date'].unique()

array(['2020-01-31T00:00:00.000000000', '2020-02-01T00:00:00.000000000',
       '2020-02-02T00:00:00.000000000', '2020-02-03T00:00:00.000000000',
       '2020-02-04T00:00:00.000000000', '2020-02-05T00:00:00.000000000',
       '2020-02-06T00:00:00.000000000'], dtype='datetime64[ns]')

In [25]:
DOS7.head()

,Fiscal Date,Item Code,Product,UPC,Bill Units,Scan Units,Est. Units On Hand,Est. Sell Thru % (Units),Bill $,Sales
0,2020-01-31,357090,ALPO SNAPS VARIETY PK DOG TRTS,1113200392,0,2,-2,-1,0,5
1,2020-01-31,336347,ALPO PRIME CUTS W/ BEEF,1113212556,0,2,-2,-1,0,10
2,2020-01-31,59442,ALPO CHOPHOUSE T-BNE STK GRAVY,1113213597,12,0,12,0,9,0
3,2020-01-31,357071,ONE CAT HAIRBALL FORMULA,1780001263,0,2,-2,-1,0,16
4,2020-01-31,357088,ONE CAT SENIOR FORUMLA,1780001885,4,0,4,0,31,0


In [26]:
n = 7

for date in list(DOS7['Fiscal Date'].unique()):
    pull = DOS7[['Sales', 'UPC']].loc[DOS7['Fiscal Date'] == date]
    parent = parent.merge(pull, how = 'left', on = 'UPC')
    parent.rename(columns = {'Sales': f'Sales_{n}'}, inplace = True)
    n -= 1
    
parent.columns

Index(['UPC', 'Bill Units 26', 'Scan Units 26', 'UOH 26', 'Sell Thru 26',
       'Bill 26', 'Sales 26', 'Bill Units 12', 'Scan Units 12', 'UOH 12',
       'Sell Thru 12', 'Bill 12', 'Sales 12', 'Bill Units 7', 'Scan Units 7',
       'UOH 7', 'Sell Thru 7', 'Bill 7', 'Sales 7', 'Sales_7', 'Sales_6',
       'Sales_5', 'Sales_4', 'Sales_3', 'Sales_2', 'Sales_1'],
      dtype='object')

In [27]:
parent.isnull().sum()

UPC                0
Bill Units 26      0
Scan Units 26      0
UOH 26             0
Sell Thru 26       0
Bill 26            0
Sales 26           0
Bill Units 12      0
Scan Units 12      0
UOH 12             0
Sell Thru 12       0
Bill 12            0
Sales 12           0
Bill Units 7       0
Scan Units 7       0
UOH 7              0
Sell Thru 7        0
Bill 7             0
Sales 7            0
Sales_7          539
Sales_6          481
Sales_5          487
Sales_4          599
Sales_3          608
Sales_2          635
Sales_1          598
dtype: int64

This is expected - we won't sell every product every day. These figures should be zeroes, since they're valid UPCs, they just didn't sell on that specific date.

In [28]:
parent.fillna(value = 0, inplace = True)
parent.isnull().sum()

UPC              0
Bill Units 26    0
Scan Units 26    0
UOH 26           0
Sell Thru 26     0
Bill 26          0
Sales 26         0
Bill Units 12    0
Scan Units 12    0
UOH 12           0
Sell Thru 12     0
Bill 12          0
Sales 12         0
Bill Units 7     0
Scan Units 7     0
UOH 7            0
Sell Thru 7      0
Bill 7           0
Sales 7          0
Sales_7          0
Sales_6          0
Sales_5          0
Sales_4          0
Sales_3          0
Sales_2          0
Sales_1          0
dtype: int64

In [29]:
parent.head()

,UPC,Bill Units 26,Scan Units 26,UOH 26,Sell Thru 26,Bill 26,Sales 26,Bill Units 12,Scan Units 12,UOH 12,...,Sell Thru 7,Bill 7,Sales 7,Sales_7,Sales_6,Sales_5,Sales_4,Sales_3,Sales_2,Sales_1
0,1113200392,240,237,3,99,638,631,104.0,105.0,-1.0,...,250.0,11.0,27.0,5.0,5.0,3.0,0.0,5.0,3.0,5.0
1,1113212556,86,84,2,98,412,400,46.0,45.0,1.0,...,100.0,29.0,27.0,10.0,0.0,5.0,0.0,5.0,7.0,0.0
2,1113213597,468,490,-22,105,365,382,168.0,197.0,-29.0,...,33.0,28.0,9.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0
3,1113214544,64,63,1,98,511,504,29.0,29.0,0.0,...,133.0,24.0,32.0,0.0,8.0,16.0,8.0,0.0,0.0,0.0
4,1780001260,44,45,-1,102,505,517,16.0,16.0,0.0,...,-1.0,0.0,23.0,0.0,11.0,11.0,0.0,0.0,0.0,0.0


# Feature Engineering - Days Since Last Sale

A common feature we'll look at in our determination is how many days we've had since a sale. Sometimes if the riser needs to be run, we'll only see a couple days of sales missing - other times, we may see periods greater than a week. Regardless, we can use our data from BI to recreate this feature, even if we can't access the data directly from SRS.

To start I'm going to take a subset of our dataset to run a few tests on, then create a helper function. I'm going to use the helper function on the subset to generate an array of numbers and then append that array to the parent dataset as the final "Days Since Last Sale" value.

In [30]:
days = parent[['Sales_1', 'Sales_2', 'Sales_3', 'Sales_4', 'Sales_5', 'Sales_6', 'Sales_7']]

In [31]:
test = days.loc[7].values

In [32]:
test

array([0., 0., 0., 0., 0., 0., 0.])

In [33]:
def address(sales):
    hist = list(sales)
    for i in hist:
        q = 7
        if i > 0:
            q = hist.index(i)
            break
        else:
            continue
    return(q)

In [34]:
address(test)

7

In [35]:
a = []

for i in list(range(days.shape[0])):
    sales = days.loc[i].values
    result = address(sales)
    a.append(result)

print(a)

[0, 1, 5, 3, 4, 2, 4, 7, 3, 1, 0, 2, 7, 1, 4, 1, 3, 0, 0, 0, 0, 6, 1, 0, 0, 1, 0, 1, 1, 0, 0, 5, 0, 0, 2, 0, 4, 4, 5, 7, 0, 7, 5, 0, 2, 1, 0, 5, 1, 3, 0, 1, 1, 1, 1, 0, 0, 7, 4, 0, 0, 2, 0, 0, 0, 6, 0, 0, 3, 0, 0, 1, 1, 5, 1, 0, 4, 2, 2, 3, 7, 0, 4, 3, 2, 4, 0, 0, 1, 6, 7, 2, 0, 0, 3, 0, 2, 7, 5, 4, 0, 0, 3, 0, 3, 1, 1, 0, 0, 1, 4, 4, 7, 1, 2, 2, 0, 5, 3, 4, 3, 1, 0, 1, 0, 2, 2, 4, 2, 5, 2, 7, 4, 0, 7, 2, 3, 0, 1, 0, 4, 4, 2, 6, 1, 1, 7, 0, 3, 4, 1, 7, 4, 7, 1, 1, 1, 5, 1, 1, 5, 0, 0, 5, 4, 4, 3, 7, 3, 0, 1, 1, 1, 1, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 6, 0, 0, 5, 0, 0, 3, 2, 0, 0, 0, 5, 4, 1, 0, 0, 0, 0, 7, 3, 4, 2, 0, 0, 1, 3, 0, 1, 0, 4, 0, 2, 2, 2, 0, 1, 0, 2, 6, 1, 1, 0, 5, 0, 1, 1, 1, 1, 4, 1, 1, 0, 7, 3, 2, 0, 1, 7, 0, 4, 1, 1, 1, 2, 3, 3, 5, 0, 7, 0, 2, 4, 1, 0, 0, 3, 1, 0, 2, 0, 0, 0, 0, 2, 7, 0, 0, 7, 7, 7, 7, 7, 3, 3, 0, 0, 0, 0, 5, 0, 6, 0, 2, 3, 2, 1, 0, 0, 7, 7, 7, 5, 7, 2, 4, 1, 4, 1, 1, 2, 0, 0, 1, 0, 7, 4, 1, 4, 1, 2, 0, 0, 0, 7, 4, 5, 0, 6, 0, 0, 3, 2, 0, 2, 5, 2, 0, 7, 6, 

In [36]:
parent['Days Since Last Sale'] = a
parent.head()

,UPC,Bill Units 26,Scan Units 26,UOH 26,Sell Thru 26,Bill 26,Sales 26,Bill Units 12,Scan Units 12,UOH 12,...,Bill 7,Sales 7,Sales_7,Sales_6,Sales_5,Sales_4,Sales_3,Sales_2,Sales_1,Days Since Last Sale
0,1113200392,240,237,3,99,638,631,104.0,105.0,-1.0,...,11.0,27.0,5.0,5.0,3.0,0.0,5.0,3.0,5.0,0
1,1113212556,86,84,2,98,412,400,46.0,45.0,1.0,...,29.0,27.0,10.0,0.0,5.0,0.0,5.0,7.0,0.0,1
2,1113213597,468,490,-22,105,365,382,168.0,197.0,-29.0,...,28.0,9.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,5
3,1113214544,64,63,1,98,511,504,29.0,29.0,0.0,...,24.0,32.0,0.0,8.0,16.0,8.0,0.0,0.0,0.0,3
4,1780001260,44,45,-1,102,505,517,16.0,16.0,0.0,...,0.0,23.0,0.0,11.0,11.0,0.0,0.0,0.0,0.0,4


In [ ]:
parent.to_pickle('parent_f.pkl')